#Telecom Domain Read & Write Ops Assignment - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS telecom_catalog_assign;
CREATE SCHEMA IF NOT EXISTS telecom_catalog_assign.landing_zone;
CREATE VOLUME IF NOT EXISTS telecom_catalog_assign.landing_zone.landing_vol;

In [0]:
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower")

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID 
102,Meera,45,Bangalore,POSTPAID 
103,Irfan,29,Hyderabad,PREPAID 
104,Raj,52,Mumbai,POSTPAID 
105,,27,Delhi,PREPAID 
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 
101\t320\t1500\t20 
102\t120\t4000\t5 
103\t540\t600\t52 
104\t45\t200\t2 
105\t0\t0\t0 
'''

tower_logs_region1 = '''
event_id|customer_id|tower_id|signal_strength|timestamp 
5001|101|TWR01|-80|2025-01-10 10:21:54 
5004|104|TWR05|-75|2025-01-10 11:01:12 
'''

tower_logs_region2 = '''
event_id|customer_id|tower_id|signal_strength|timestamp 
5002|102|TWR02|-85|2025-01-10 10:21:54 
5005|106|TWR06|-76|2025-01-10 11:01:22 
'''

##2. Filesystem operations
1. Write dbutils.fs code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",customer_csv, True)

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv",usage_tsv, True)

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_log_region1.csv",tower_logs_region1, True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/tower_log_region2.csv",tower_logs_region2, True)

In [0]:
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/"))
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"))
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/"))
print(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"))

##3. Spark Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
tower_df = spark.read.csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2"], header=True, inferSchema=True, recursiveFileLookup=True, sep="|",pathGlobFilter="*.csv")
display(tower_df)


In [0]:
tower_df2 = spark.read.csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*"], header=True, inferSchema=True, recursiveFileLookup=True, sep="|",pathGlobFilter="*.csv")
display(tower_df2)

In [0]:
tower_df3 = spark.read.options(recursiveFileLookup="true", sep="|",header=True,inferschema=True, pathGlobFilter="*.csv").csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/","/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2/"])
display(tower_df3)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
cust_df = spark.read.options(header=False, inferSchema=False).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df)

In [0]:
cust_df1 = spark.read.options(header=True, inferSchema=True).format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")
display(cust_df1)

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
cust_col = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=False, inferSchema=False).toDF("Cust_id", "Cust_name", "Age", "City", "Plan")
display(cust_col)

In [0]:
usage_tble = "customer_id int,voice_mins int,data_mb int,sms_count int"
usage_df = spark.read.schema(usage_tble).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv", header=True, sep="\t",inferSchema=False)
usage_df.printSchema()
display(usage_df)

In [0]:


from pyspark.sql.types import *
custom_schema = StructType(
    [StructField("event_id", IntegerType(),True),
     StructField("customer_id", IntegerType(), True),
     StructField("tower_id", StringType(), True),
     StructField("signal_strength", IntegerType(), True),
     StructField("timestamp", TimestampType(), True)
     ])

tower_dfs = spark.read.schema(custom_schema).options(header= True, sep="|",pathGlobFilter="*.csv", recursiveFileLookup=True).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*")
display(tower_dfs)


## Spark Write Operations using 
- csv, json, orc, parquet, delta, saveAsTable, insertInto, xml with different write mode, header and sep options

##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1
df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/", header=True, inferSchema=True)

write_df = df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/write_customer", header=True, sep="~", mode="overwrite")

#2
df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/", header=True, inferSchema=True)

write_df = df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/write_usage", header=True, mode="Append")

#3
df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*", pathGlobFilter="*.csv", recursiveFileLookup=True, header=True, inferSchema=True)

write_df = df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/write_tower", header=True, sep="|", mode="overwrite")
display(write_df)

#4
df_tower = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*", pathGlobFilter="*.csv", recursiveFileLookup=True, )
display(df_tower)

#5
#i can able to download and see the files opening in notepad++

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1
df_cust_json = df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/write_customer", mode="overwrite")
#2
df_usage_json = df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/json_usage", mode="Append", compression="snappy")
#3
df_tower_json = df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/json_tower", mode="Ignore")
#4
display(df1.limit(5))
#5
# i can see the files downloaded and opened in notepad++



##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1
custdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True, inferSchema=True)

custdataframe.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/parquetcustomer/", mode="overwrite", compression="gzip")

#2
usagedataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/", header=True, inferSchema=True, sep="\t")

usagedataframe.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/parquetusage/", mode="overwrite")

#3
towerdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*", pathGlobFilter="*.csv", recursiveFileLookup=True, header=True, inferSchema=True, sep="|")

towerdataframe.write.mode("overwrite").option("compression", "gzip").parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/parquettower/")

#4
display(usagedataframe.limit(5))

#5
# i can see the files downloaded and opened in notepad++

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
#1
custdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True, inferSchema=True).toDF("Cust_id", "Cust_name", "Age", "City", "Plan")

custdataframe.write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/orccustomer/", mode="overwrite")

#2
usagedataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/", header=True, inferSchema=True, sep="\t")

usagedataframe.write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/orcusage/", mode="append")

#3
towerdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*", pathGlobFilter="*.csv", recursiveFileLookup=True, header=True, inferSchema=True, sep="|")

orc_df = towerdataframe.write.mode("overwrite").orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/orctower/")

#4
display(towerdataframe)

#5
# i can see the files downloaded and opened in notepad++


##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
#1
custdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True, inferSchema=True).toDF("Cust_id", "Cust_name", "Age", "City", "Plan")

custdataframe.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltacustomer", mode="overwrite")

spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/deltacustomer").show(2)

#2
usagedataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/", header=True, inferSchema=True, sep="\t").toDF("customer_id", "voice_mins", "data_mb", "sms_count")

usagedataframe.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltausage", mode='append')

spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/deltausage").show(2)

#3
towerdataframe = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*", pathGlobFilter="*.csv", recursiveFileLookup=True, header=True, inferSchema=True, sep="|").toDF("event_id", "customer_id", "tower_id", "signal_strength", "timestamp")

towerdataframe.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/deltatower")

spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/deltatower").show(2)

#4
display(usagedataframe)

#5
# i can see the files downloaded and opened in notepad++


##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.


In [0]:
#1
custdataframe.write.mode("overwrite").saveAsTable("default.customer_table")
spark.read.table("default.customer_table").show(2)
#2
usagedataframe.write.saveAsTable("default.usage_table", mode="overwrite")
spark.read.table("default.usage_table").show(2)
#4
#yes checked and it is
#5
spark.sql(" select * from default.usage_table").show(2)




##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

In [0]:
#1 inserted tower data to customer table
towerdataframe.write.insertInto("customer_table")

spark.sql("select * from default.customer_table").show()

#2
usagedataframe.write.insertInto("usage_table", overwrite=True)


##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
custdataframe.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/xmlcustomer/", rowTag="customer")

usagedataframe.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/xmlusage/", rowTag="usage")



##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml) 
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.

- CSV = 173 b 
- Json = 206 b 
- xml = 931 b
- Orc = 942 b
- delta = 1.53 kb
- Parquet = 1.86 kb



##15. Do a final exercise of defining one/two liner of... 
1. When to use/benifits csv
2. When to use/benifits json
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta
6. When to use/benifit xml
7. When to use/benifit delta tables
